In [1]:
#!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import string
import json
import re

In [2]:
ALL_COLORS = [1, 2, 3, 4]  
DIC_COLORS = {1: 'Rouge', 2: 'Blanc', 3: 'Rosé', 4: 'Ambré'}
KNOWN_PAGES = [3167, 2590, 318, 13]
COLOR = 1
MIN_PAGE = 400  # Attention : commence à zéro
MAX_PAGE = 599

class MaClasseScrapy(scrapy.Spider) :
    name = 'Scraping bettanedesseauve version 0.01'

    start_urls = ["https://www.mybettanedesseauve.fr/advanced-search/"]

    def parse(self, response):
        yield scrapy.FormRequest(
            "https://www.mybettanedesseauve.fr/display-result/",  # champ 'action' du formulaire
            callback=self.parse_first_page,
            method='POST',
            formdata={
                'colors': str(COLOR),
                'appellations': "",
                'regions': "",
                'wineTypes': "",
                'vintages': "",
                'prices': "",
                'events': "",
                'query': "",
                'bdse_params-page_source': '2',
                'filtre': 'enable'
            }
        )

    def parse_first_page(self, response):
        logging.info("###### (parse_first_page) URL : " + response.url)
        for pagenum in range(MIN_PAGE, MAX_PAGE + 1):
            yield scrapy.Request(
                "https://www.mybettanedesseauve.fr/display-result/?pa=" + str(pagenum),
                callback = self.parse_page,
                cb_kwargs={'pagenum': pagenum}
            )

    def parse_page(self, response, pagenum):
        logging.info("###### (parse_page) URL : " + response.url)
        for zone in response.css("ul.table-hover"):
            for r in zone.css("a"):
                country = None
                terroir = None
                origin = r.css("h4.bdt-search-engine::text").get()
                if origin!=None:
                    country = re.sub(r"\,[^\,]*$", "", origin).replace('\n', '')
                    terroir = found.group(1).strip() if (found := re.search(r"\-([^\-]*)$", origin))!=None else None

                wine_url = response.urljoin(r.attrib['href'])
                yield scrapy.Request(
                    wine_url,
                    callback = self.parse_wine,
                    cb_kwargs={'country': country, 'terroir': terroir, 'pagenum': pagenum}
                )
             
    def parse_wine(self, response, country, terroir, pagenum):
        logging.info("###### (parse_wine) URL : " + response.url)
        data = {'color': DIC_COLORS[COLOR], 'page': pagenum, 'country': country}
        
        if terroir!=None:
            data['terroir'] = terroir

        for zone in response.css("div.td-page-content"):
            r_domain = zone.css("div h3::text")
            if r_domain!=None:
                data['domain_name'] = r_domain.get()
            
            r_appellation = zone.css("div h4::text")
            if r_appellation!=None:
                data['appellation'] = r_appellation.get()

            r_various = zone.css("div h5::text")
            if r_various!=None:
                L_various = r_various.get().split(',')
                L_various = [L_various[len(L_various) - i - 1] for i in range(len(L_various))]
                data['vintage'] = L_various[0]
                if len(L_various)>1:
                    data['wine_type'] = L_various[1]
                    if len(L_various)>2:
                        data['wine_name'] = L_various[2]
            
            r_score = zone.css("div.row div.col div.row div.col div span::text")
            if r_score!=None:
                data['score'] = re.sub(r"\/.*$", '', r_score.get())
            
            r_various = zone.css("div div p::text")
            if r_various!=None:
                L_various = r_various.getall()
                if len(L_various)>1:
                    data['comment'] = L_various[1]
                if len(L_various)>2:
                    data['keep_comment'] = L_various[2]

            r_stars = zone.css("div div div span")
            stars = 0
            if r_stars!=None:
                #stars = sum([1 for t in r_stars.getall()])
                stars = sum([1 if (('style' in r.attrib.keys()) and (r.attrib['style'].strip()==r"text-align: center ;font-size: xx-large;color: red")) else 0 for r in r_stars])
            data['stars'] = stars

            yield data


# ATTENTION : à précéder d’un restart kernel :
def scrape_all(filepath):
    process = CrawlerProcess(settings={
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': logging.INFO,
        'FEEDS': {filepath : {"format": "json"}},
        'AUTOTHROTTLE_ENABLED': True 
    })
    process.crawl(MaClasseScrapy)
    process.start()

import time
filename = 'bettane_0.01_output/scrapy_bettane_' + DIC_COLORS[COLOR] + '_' + str(MIN_PAGE) + 'to' + str(MAX_PAGE) + '_' + str(int(time.time())) + '.json'
scrape_all(filename)


spider_output
    for x in result:
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\referer.py", line 340, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "C:\ProgramData\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 62,

In [3]:
# Essai de chargement dans Pandas :

import pandas as pd
DF_test = pd.read_json(filename)
print(DF_test.info())

DF_test.head(30)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   color         462 non-null    object 
 1   page          462 non-null    int64  
 2   country       462 non-null    object 
 3   terroir       342 non-null    object 
 4   domain_name   462 non-null    object 
 5   appellation   462 non-null    object 
 6   vintage       462 non-null    int64  
 7   wine_type     462 non-null    object 
 8   score         462 non-null    float64
 9   comment       462 non-null    object 
 10  keep_comment  459 non-null    object 
 11  stars         462 non-null    int64  
 12  wine_name     95 non-null     object 
dtypes: float64(1), int64(3), object(9)
memory usage: 47.0+ KB
None


,color,page,country,terroir,domain_name,appellation,vintage,wine_type,score,comment,keep_comment,stars,wine_name
0,Rouge,418,France,Perrières grand cru,Les Parcellaires de Saulx,Corton - Perrières grand cru,2017,Sec,17.0,"Il évoque la ronce, il a de l’élégance, de la ...",A boire entre 2023 et 2029,1,NaN
1,Rouge,417,France,Bressandes grand cru,Domaine Buisson-Charles,Corton - Bressandes grand cru,2017,Sec,17.0,"Notes de tabac blond, la bouche a cette rondeu...",A boire entre 2022 et 2024,3,NaN
2,Rouge,416,France,Jacques,Domaine Armand Rousseau,Gevrey-Chambertin premier cru,2017,Sec,17.0,A boire avant 2028,Éric Rousseau dispose d’un patrimoine de vigne...,5,NaN
3,Rouge,415,France,Georges,Domaine Philippe Gavignet,Nuits-Saint-Georges,2017,Sec,17.0,"La structure impressionne, son équilibre aussi...",A boire avant 2030,2,NaN
4,Rouge,414,France,Chambertin grand cru,Domaine René Bouvier,Charmes-Chambertin grand cru,2017,Sec,17.0,"Note de poivre blanc, de groseille, très mazoy...",A boire entre 2021 et 2030,4,NaN
5,Rouge,413,France,Les Chaumes,Domaine François Lamarche,Vosne-Romanée premier cru,2017,Sec,17.0,Le domaine cherche à produire un chaumes sur l...,A boire entre 2022 et 2030,4,NaN
6,Rouge,412,France,Clos Jus,Domaine Ragot,Givry premier cru,2017,Sec,17.0,"Un fruit élégant, une vraie profondeur, un tan...",A boire avant 2022,3,NaN
7,Rouge,411,France,Émilion grand cru,L’If,Saint-Émilion grand cru,2017,Sec,17.0,A boire entre 2022 et 2041,Cette propriété bien mise en valeur depuis 201...,3,NaN
8,Rouge,410,France,NaN,Château Clerc Milon,Pauillac,2017,Sec,17.0,"Riche et onctueux, tannin ferme mais sans dure...",A boire entre 2027 et 2037,3,NaN
9,Rouge,409,France,NaN,Caveau de Bacchus,Arbois,2017,Sec,17.0,Assemblage des parcelles Ruzard et Rosière (mi...,A boire avant 2024,3,trousseau Cuvée des Géologues Ruzard-Rosières


In [4]:
""

''

In [5]:
""


''